In [ ]:
import sqlite3
import os

# --- Configuration and Initialization ---
DB_NAME = 'library.db'

def create_table():
    """Initializes the database and creates the 'books' table."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    # Create the table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS books (
            id INTEGER PRIMARY KEY,
            title TEXT NOT NULL,
            author TEXT NOT NULL,
            isbn TEXT UNIQUE NOT NULL,
            status TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

# --- NEW FUNCTION: Insert Sample Data ---
def insert_sample_books():
    """Inserts a few books if the database is currently empty."""
    sample_books = [
        ("The Lord of the Rings", "J.R.R. Tolkien", "9780618260214", "Available"),
        ("Python Crash Course", "Eric Matthes", "9781593279282", "Available"),
        ("Data Science Handbook", "Field Cady", "9781119092955", "Available"),
        ("1984", "George Orwell", "9780451524935", "Available"),
    ]

    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    # Check if the table is empty
    cursor.execute("SELECT COUNT(*) FROM books")
    if cursor.fetchone()[0] == 0:
        print("\n--- Inserting Sample Books for easy testing ---")
        try:
            cursor.executemany(
                "INSERT INTO books (title, author, isbn, status) VALUES (?, ?, ?, ?)",
                sample_books
            )
            conn.commit()
            print("Successfully added 4 sample books.")
        except sqlite3.IntegrityError:
             # This handles the unlikely case where the sample ISBNs might conflict during setup
             pass

    conn.close()


# Initialize the database table when the script starts
create_table()
# Call the new function to populate the data
insert_sample_books()


# =======================================================
# Module 1: Database Manager (CRUD Operations)
# =======================================================

def add_book(title, author, isbn):
    """CREATE: Adds a new book to the database. Status defaults to 'Available'."""
    try:
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        cursor.execute("INSERT INTO books (title, author, isbn, status) VALUES (?, ?, ?, ?)",
                       (title, author, isbn, 'Available'))
        conn.commit()
        conn.close()
        return True
    except sqlite3.IntegrityError:
        return False # ISBN already exists
    except Exception as e:
        print(f"Database error: {e}")
        return False

def get_all_books():
    """READ: Retrieves all books from the database."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT id, title, author, isbn, status FROM books")
    books = cursor.fetchall()
    conn.close()
    return books

def get_book_by_id(book_id):
    """READ: Retrieves a single book by its ID."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT id, title, author, isbn, status FROM books WHERE id = ?", (book_id,))
    book = cursor.fetchone()
    conn.close()
    return book

def update_book_status(book_id, new_status):
    """UPDATE: Modifies the status of a book (e.g., 'Checked Out')."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("UPDATE books SET status = ? WHERE id = ?", (new_status, book_id))
    conn.commit()
    rows_affected = cursor.rowcount
    conn.close()
    return rows_affected > 0

def delete_book(book_id):
    """DELETE: Removes a book from the database."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM books WHERE id = ?", (book_id,))
    conn.commit()
    rows_affected = cursor.rowcount
    conn.close()
    return rows_affected > 0

# =======================================================
# Module 2: Library Logic
# =======================================================

def check_out_book(book_id):
    """Attempts to check out a book if it is 'Available'."""
    book = get_book_by_id(book_id)
    if not book:
        return "Book ID not found."

    # Book structure: (id, title, author, isbn, status)
    current_status = book[4]

    if current_status == 'Available':
        update_book_status(book_id, 'Checked Out')
        return f"Successfully checked out: {book[1]} by {book[2]}"
    else:
        return f"Book '{book[1]}' is already {current_status}."

def return_book(book_id):
    """Attempts to mark a book as 'Available' if it is 'Checked Out'."""
    book = get_book_by_id(book_id)
    if not book:
        return "Book ID not found."

    current_status = book[4]

    if current_status == 'Checked Out':
        update_book_status(book_id, 'Available')
        return f"Successfully returned: {book[1]} by {book[2]}"
    else:
        return f"Book '{book[1]}' is currently {current_status}. Cannot return."

def list_all_books():
    """Fetches and formats the list of all books."""
    books = get_all_books()
    if not books:
        return "The library is empty."

    output = "\n--- Current Library Inventory ---\n"
    output += f"{'ID':<5} {'Title':<30} {'Author':<20} {'ISBN':<15} {'Status':<15}\n"
    output += "-" * 85 + "\n"
    for book in books:
        # Check if book[1] (title) is too long for the column
        title_display = book[1] if len(book[1]) <= 30 else book[1][:27] + '...'
        output += f"{book[0]:<5} {title_display:<30} {book[2]:<20} {book[3]:<15} {book[4]:<15}\n"
    output += "---------------------------------\n"
    return output

def remove_book(book_id):
    """Wrapper for the delete operation."""
    if delete_book(book_id):
        return "Book successfully deleted."
    else:
        return "Error: Book ID not found for deletion."

# =======================================================
# Module 3: User Interface and Workflow
# =======================================================

def display_menu():
    """Displays the main menu options to the user."""
    print("\n\n--- Library Management System Menu ---")
    print("1. Add New Book")
    print("2. View All Books")
    print("3. Check Out Book")
    print("4. Return Book")
    print("5. Delete Book")
    print("6. Exit")
    print("--------------------------------------")

def handle_add_book():
    """Workflow for adding a book."""
    print("\n--- Add New Book ---")
    # Use input() in Colab will prompt the user in the output cell
    title = input("Enter Title: ")
    author = input("Enter Author: ")
    isbn = input("Enter ISBN: ")

    if not title or not author or not isbn:
        print("Error: All fields are required.")
        return

    if add_book(title, author, isbn):
        print(f"\nSUCCESS: Book '{title}' added to the library.")
    else:
        print("\nERROR: Book could not be added. ISBN might already exist.")

def handle_checkout_return(action):
    """Workflow for checking out or returning a book."""
    try:
        book_id = int(input(f"\nEnter the ID of the book to {action}: "))

        if action == 'checkout':
            result = check_out_book(book_id)
        elif action == 'return':
            result = return_book(book_id)
        else:
            return

        print(f"\n{result}")

    except ValueError:
        print("\nERROR: Invalid input. Please enter a valid number for the Book ID.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

def handle_delete_book():
    """Workflow for deleting a book."""
    try:
        book_id = int(input("\nEnter the ID of the book to DELETE: "))
        result = remove_book(book_id)
        print(f"\n{result}")
    except ValueError:
        print("\nERROR: Invalid input. Please enter a valid number for the Book ID.")

def run_system():
    """The main application loop."""
    print("Welcome to the Colab Library Management System!")
    while True:
        display_menu()
        choice = input("Enter your choice (1-6): ")

        if choice == '1':
            handle_add_book()
        elif choice == '2':
            print(list_all_books())
        elif choice == '3':
            handle_checkout_return('checkout')
        elif choice == '4':
            handle_checkout_return('return')
        elif choice == '5':
            handle_delete_book()
        elif choice == '6':
            print("\nExiting Library System. Goodbye!")
            # Check if the database file was created
            if os.path.exists(DB_NAME):
                print(f"Note: The database file '{DB_NAME}' is saved in your Colab session.")
            break
        else:
            print("\nInvalid choice. Please enter a number between 1 and 6.")

if __name__ == "__main__":
    run_system()


--- Inserting Sample Books for easy testing ---
Successfully added 4 sample books.
Welcome to the Colab Library Management System!


--- Library Management System Menu ---
1. Add New Book
2. View All Books
3. Check Out Book
4. Return Book
5. Delete Book
6. Exit
--------------------------------------
Enter your choice (1-6): 2

--- Current Library Inventory ---
ID    Title                          Author               ISBN            Status         
-------------------------------------------------------------------------------------
1     The Lord of the Rings          J.R.R. Tolkien       9780618260214   Available      
2     Python Crash Course            Eric Matthes         9781593279282   Available      
3     Data Science Handbook          Field Cady           9781119092955   Available      
4     1984                           George Orwell        9780451524935   Available      
---------------------------------



--- Library Management System Menu ---
1. Add New Book
2. View 